In [8]:
import os
import zipfile
import torch
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from sklearn.metrics import accuracy_score


In [9]:
# Define transformations for data preprocessing
transform = transforms.Compose([
    #transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [10]:

train_dataset = ImageFolder('train', transform=transform)
test_dataset = ImageFolder('test', transform=transform)
valid_dataset = ImageFolder('valid', transform=transform)

In [11]:

# Create DataLoader for each dataset
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)


In [12]:
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self, num_classes, cl2_feature_maps, hidden_size1, hidden_size2):
        super(CNN, self).__init__()
        # Convolutional Layer 1
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=4, kernel_size=3, stride=1, padding=1)
        # Pooling Layer 1
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        # Convolutional Layer 2
        self.conv2 = nn.Conv2d(in_channels=4, out_channels=cl2_feature_maps, kernel_size=3, stride=1, padding=1)
        # Pooling Layer 2
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        # Fully Connected Layer
        self.fc = nn.Linear(cl2_feature_maps * 56 * 56, hidden_size1)  # Assuming input size after pooling is 56x56
        # Hidden Layer 1
        self.hidden1 = nn.Linear(hidden_size1, hidden_size2)
        # Hidden Layer 2
        self.hidden2 = nn.Linear(hidden_size2, num_classes)

    def forward(self, x):
        # Forward pass
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = F.tanh(self.fc(x))
        x = F.tanh(self.hidden1(x))
        x = self.hidden2(x)
        return x

# Hyperparameters
num_classes = len(train_dataset.classes)
hidden_size1 = 1000  # Size for the first hidden layer
hidden_size2 = 50   # Size for the second hidden layer
num_epochs = 15
learning_rate = 0.001
cl2_feature_maps_range = range(4, 5)  # Assuming you want to loop over feature maps from 2 to 2


In [13]:
# Training loop
for i in cl2_feature_maps_range:
    print(f"***    {i}   ********")
    cl2_feature_maps = i
    model = CNN(num_classes, cl2_feature_maps, hidden_size1, hidden_size2)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)  # Adding weight decay for regularization

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader)}")


***    4   ********
Epoch [1/15], Loss: 1.3917403627955724
Epoch [2/15], Loss: 1.1742284685846358
Epoch [3/15], Loss: 1.053607359765068
Epoch [4/15], Loss: 0.9558688818462311
Epoch [5/15], Loss: 0.8387132201875959
Epoch [6/15], Loss: 0.6429070612740895
Epoch [7/15], Loss: 0.44443037254469736
Epoch [8/15], Loss: 0.29382025060199557
Epoch [9/15], Loss: 0.18809460387343452
Epoch [10/15], Loss: 0.10655562224842254
Epoch [11/15], Loss: 0.0597305851027606
Epoch [12/15], Loss: 0.0703100785908718
Epoch [13/15], Loss: 0.07679041244444393
Epoch [14/15], Loss: 0.0791295241741907
Epoch [15/15], Loss: 0.14218949816293186


In [14]:
# Testing and evaluation
def calculate_accuracy(loader):
    model.eval()
    correct = 0
    total = 0
    predictions = []
    true_labels = []
    with torch.no_grad():
        for images, labels in loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            predictions.extend(predicted.numpy())
            true_labels.extend(labels.numpy())
    accuracy = correct / total
    return accuracy, np.array(predictions), np.array(true_labels)

for i in cl2_feature_maps_range:
    cl2_feature_maps = i
    print(f"Cl2 Feature Maps: {cl2_feature_maps}")
    
    # Training accuracy
    train_accuracy, _, _ = calculate_accuracy(train_loader)
    print(f"Training Accuracy: {train_accuracy}")

    # Validation accuracy
    valid_accuracy, _, _ = calculate_accuracy(valid_loader)
    print(f"Validation Accuracy: {valid_accuracy}")

    # Testing accuracy
    test_accuracy, _, _ = calculate_accuracy(test_loader)
    print(f"Test Accuracy: {test_accuracy}")


Cl2 Feature Maps: 4
Training Accuracy: 0.995
Validation Accuracy: 0.524
Test Accuracy: 0.504
